Mid term Report title


In [ ]:
# Import libraries and modules
import yfinance as yf
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import json
import os


import matplotlib.pyplot as plt
from matplotlib import style

# Show all matplotlib graphs inline
%matplotlib inline  
# using the style for the plot
plt.style.use('dark_background')
# set plot size
plt.rcParams['figure.figsize'] = [30, 15]


In [ ]:
#Scope Limits
StartTime = "2012-05-01"
EndTime = "2022-05-01"
TweetLimit = 100

# scope of a jump
JumpTriggerRatio = 1
JumpTriggerPercentage = 5
MinDaysBetweenJumps = 7


# caching folders
TweetCacheFolder = "CachedTweets"
StockCacheFolder = "CachedStockData"

Collect Twitter Data

In [ ]:
#what users do we care about

# names from
# https://en.wikipedia.org/wiki/List_of_most-followed_Twitter_accounts

with open("TwitterUserNames.json", "r") as file:
	usernames = json.load(file)

In [ ]:
#get tweets from users
def GetTweetsFromUser(username):

	if not os.path.isdir(TweetCacheFolder):
		os.mkdir(TweetCacheFolder)

	cachePath = os.path.join(TweetCacheFolder, "tweets_" + username + ".csv")

	# cache exists
	if os.path.exists(cachePath):
		df = pd.read_csv(cachePath)

	else:
		query = "(from:" + username + ") since:" + StartTime + " until:" + EndTime
		
		tweets = []
		for tweet in sntwitter.TwitterSearchScraper(query).get_items():

			if len(tweets) == TweetLimit:
				break
			else:
				# print(tweet.json())
				#break
				tweets.append([tweet.date, tweet.user.username, tweet.content, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount, tweet.id])
		
		df = pd.DataFrame(tweets, columns=['Date', 'User', 'Content', 'ReplyCount', 'RetweetCount', 'LikeCount', 'QuoteCount', 'TweetID'])

		# to save to csv as a cache
		df.to_csv(cachePath)
	

	return df

tweetsDf = GetTweetsFromUser(usernames[0])

In [ ]:
for username in usernames:
	tweetsDf = GetTweetsFromUser(username)


Collect Stock Market Data

In [ ]:
#load stocks to track
with open("StockTags_SnP.json", "r") as file:
	stockTags = json.load(file)


In [ ]:
def LoadStockData(stockTag):

	if not os.path.isdir(StockCacheFolder):
		os.mkdir(StockCacheFolder)

	cachePath = os.path.join(StockCacheFolder, "Stock_" + stockTag + ".csv")

	# cache exists
	if os.path.exists(cachePath):
		stockDataDf = pd.read_csv(cachePath)

	else:
		stockDataDf = yf.download(stockTag, start=StartTime, end=EndTime, progress=False)

		#save data to local cache file
		stockDataDf.to_csv(cachePath)

	return stockDataDf

In [ ]:

for stockTag in stockTags:
	LoadStockData(stockTag)


Now time to review the data downloaded

In [ ]:
def CreateMovingAverage(df, columnToAverage, averageLength):

	movingAverage = []
	movingAverageUpperQuantile = []
	movingAverageLowerQuantile = []

	rows = df.loc[:,columnToAverage]
	for i in range(len(rows)):

		# get window
		if i < averageLength:
			window = [0]
		else:
			window = rows[i - averageLength : i]

		# calculate moving avg
		windowAverage = round(sum(window) / averageLength, 2)
		movingAverage.append(windowAverage)

		# calculate UpperQuantile
		upperQuantile = np.quantile(window, 0.9)
		movingAverageUpperQuantile.append(upperQuantile)

		# calculate LowerQuantile
		lowerQuantile = np.quantile(window, 0.1)
		movingAverageLowerQuantile.append(lowerQuantile)

	df.insert(len(df.columns), columnToAverage + "MA_" + str(averageLength), movingAverage, allow_duplicates=True)
	df.insert(len(df.columns), columnToAverage + "MA_" + str(averageLength) + "_UpperQuantile", movingAverageUpperQuantile, allow_duplicates=True)
	df.insert(len(df.columns), columnToAverage + "MA_" + str(averageLength) + "_LowerQuantile", movingAverageLowerQuantile, allow_duplicates=True)
	return

In [ ]:
def DetectValueJumping(df, column, lastValueColumn, currentValueColumn):
	isValueJumping = []

	for i in range(len(df)):

		# get window
		if i < 45: #skip the first few days as the moving averages take a few days to settle
			isJump = False
		else:
			upperQuantile = df[column + "MA_7MA_30_UpperQuantile"][i]
			lowerQuantile = df[column + "MA_7MA_30_LowerQuantile"][i]
			lastValue = df[lastValueColumn][i]
			currentValue = df[currentValueColumn][i]

			interQuantileRange = upperQuantile - lowerQuantile
			triggerRange = JumpTriggerRatio * interQuantileRange

			meantRatio = currentValue > upperQuantile + triggerRange or currentValue < lowerQuantile - triggerRange

			meantPercentage = False
			if lastValue != 0:
				meantPercentage = abs(lastValue - currentValue) / lastValue > JumpTriggerPercentage / 100
			
			isJump = meantRatio and meantPercentage

		isValueJumping.append(isJump)

	df.insert(len(df.columns), column + "_IsJumping", isValueJumping, allow_duplicates=True)

	return

def CreateStockTrendData(stockData):

	CreateMovingAverage(stockData, "Volume", 3)
	CreateMovingAverage(stockData, "Volume", 7)
	CreateMovingAverage(stockData, "VolumeMA_7", 30)

	CreateMovingAverage(stockData, "Open", 3)
	CreateMovingAverage(stockData, "Open", 7)
	CreateMovingAverage(stockData, "OpenMA_7", 30)

	DetectJumps(stockData)

	return

def DetectJumps(stockData):

	DetectValueJumping(stockData, "Volume", "VolumeMA_7", "VolumeMA_3")
	DetectValueJumping(stockData, "Open", "OpenMA_3", "Open")

	
	bothJumped = stockData["Volume_IsJumping"] * stockData["Open_IsJumping"]
	stockData.insert(len(stockData.columns), "Both_IsJumping", bothJumped, allow_duplicates=True)

	eitherJumped = stockData["Volume_IsJumping"] + stockData["Open_IsJumping"]
	stockData.insert(len(stockData.columns), "Either_IsJumping", eitherJumped, allow_duplicates=True)


In [ ]:
for stockTag in stockTags:
	stockData = LoadStockData(stockTag)
	CreateStockTrendData(stockData)
	

In [ ]:
numCharts = 6

fig, axs = plt.subplots(numCharts, sharex=True)
fig.suptitle("Open Price / Time")

for i in range(numCharts):
	stockData = LoadStockData(stockTags[i])
	CreateStockTrendData(stockData)

	axs[i].set_title(stockTags[i])
	axs[i].plot(stockData["Date"], stockData["Open"], label = "Open")
	# axs[i].plot(stockData["Date"], stockData["OpenMA_7MA_30_UpperQuantile"], 'r--', label = "OpenMA_7MA_30_UpperQuantile")
	# axs[i].plot(stockData["Date"], stockData["OpenMA_7MA_30_LowerQuantile"], 'g--', label = "OpenMA_7MA_30_LowerQuantile")
	
	axs[i].set(xlabel='Date', ylabel='Open Price')
	axs[i].label_outer()

	ax2 = axs[i].twinx()
	# ax2.bar(stockData["Date"], stockData["Open_IsJumping"], label="open jump")
	ax2.bar(stockData["Date"], stockData["Both_IsJumping"], label="both jump")

	ax2.legend()
plt.show()

In [ ]:
numCharts = 6

fig, axs = plt.subplots(numCharts, sharex=True)
fig.suptitle("Volume / Time")

for i in range(numCharts):
	stockData = LoadStockData(stockTags[i])
	CreateStockTrendData(stockData)

	axs[i].set_title(stockTags[i])
	axs[i].plot(stockData["Date"], stockData["VolumeMA_7"], label = "VolumeMA_7")
	# axs[i].plot(stockData["Date"], stockData["VolumeMA_7MA_30_UpperQuantile"], 'r--', label = "VolumeMA_7MA_30_UpperQuantile")
	# axs[i].plot(stockData["Date"], stockData["VolumeMA_7MA_30_LowerQuantile"], 'g--', label = "VolumeMA_7MA_30_LowerQuantile")
	
	axs[i].set(xlabel='Date', ylabel='Volume')
	axs[i].label_outer()

	ax2 = axs[i].twinx()
	
	# ax2.bar(stockData["Date"], stockData["Volume_IsJumping"], label="volume jump")
	ax2.bar(stockData["Date"], stockData["Both_IsJumping"], label="both jump")

	ax2.legend()

plt.show()

now we need to get the dates that we care about

In [ ]:
def GetStockJumpsDates(stockData, jumpColumn):

	dates = []
	daysSinceLastJump = MinDaysBetweenJumps

	for i in range(len(stockData)):
		daysSinceLastJump += 1

		isJump = stockData[jumpColumn][i]

		if isJump and daysSinceLastJump >= MinDaysBetweenJumps:
			dates.append(stockData["Date"][i])
			daysSinceLastJump = 0

	return dates

In [ ]:
# how many jump dates are there between all the stocks

bothSet = set()
volumeSet = set()
openSet = set()

for stockTag in stockTags:
	stockData = LoadStockData(stockTag)
	CreateStockTrendData(stockData)

	bothSet.update(GetStockJumpsDates(stockData, "Both_IsJumping"))
	volumeSet.update(GetStockJumpsDates(stockData, "Volume_IsJumping"))
	openSet.update(GetStockJumpsDates(stockData, "Open_IsJumping"))


print("number of Open jumps: " + str(len(openSet)))
print("number of Volume jumps: " + str(len(volumeSet)))
print("number of Both jumps: " + str(len(bothSet)))

In [ ]:
for stockTag in stockTags:
	stockData = LoadStockData(stockTag)
	CreateStockTrendData(stockData)

	dates = GetStockJumpsDates(stockData, "Both_IsJumping")
